# Applying Naive Bayes Classifier on Movie Review Dataset

### Muhammad Arham
### 10016866912

### Parsing & Loading the Dataset

In [54]:
#https://stackoverflow.com/questions/18262293/how-to-open-every-file-in-a-folder
#---------------------------------------------------------------------------------------------------------#
#Loading Train Data
#importing from the pos folder
cwd = "C:\\Users\\Arham\\Documents\\UTA\\SPRING 2020\\DM\\Project\\aclImdb_v1.tar\\aclImdb_v1\\aclImdb\\train\\pos"

import os
import pandas as pd 

row = []
for filename in os.listdir(cwd):
    with open(os.path.join(cwd, filename), 'r', encoding="utf8") as f: # open in readonly mode
        #print (filename + "," + f.read() + "," + "1")
        col = []
        col.append(filename)
        col.append(f.read())
        col.append("1")
        row.append(col)

#---------------------------------------------------------------------------------------------------------#
#importing from the neg folder
cwd = "C:\\Users\\Arham\\Documents\\UTA\\SPRING 2020\\DM\\Project\\aclImdb_v1.tar\\aclImdb_v1\\aclImdb\\train\\neg"               
for filename in os.listdir(cwd):
    with open(os.path.join(cwd, filename), 'r', encoding="utf8") as f: # open in readonly mode
        #print (filename + "," + f.read() + "," + "1")
        col = []
        col.append(filename)
        col.append(f.read())
        col.append("0")
        row.append(col)


df = pd.DataFrame(row,columns =['File','Review','Class'])


df.head()

#Class 1 represents all the reviews from the positive foler and Class 0 represents reviews from negative folder

#----------------------------------------------------------------------------------------------
#Loading Test data

cwd = "C:\\Users\\Arham\\Documents\\UTA\\SPRING 2020\\DM\\Project\\aclImdb_v1.tar\\aclImdb_v1\\aclImdb\\test\\pos"

row = []
for filename in os.listdir(cwd):
    with open(os.path.join(cwd, filename), 'r', encoding="utf8") as f: # open in readonly mode
        #print (filename + "," + f.read() + "," + "1")
        col = []
        col.append(filename)
        col.append(f.read())
        col.append("1")
        row.append(col)

#---------------------------------------------------------------------------------------------------------#
#importing from the neg folder
cwd = "C:\\Users\\Arham\\Documents\\UTA\\SPRING 2020\\DM\\Project\\aclImdb_v1.tar\\aclImdb_v1\\aclImdb\\test\\neg"               
for filename in os.listdir(cwd):
    with open(os.path.join(cwd, filename), 'r', encoding="utf8") as f: # open in readonly mode
        #print (filename + "," + f.read() + "," + "1")
        col = []
        col.append(filename)
        col.append(f.read())
        col.append("0")
        row.append(col)


test_df = pd.DataFrame(row,columns =['File','Review','Class'])

#test_df.head()

#Class 1 represents all the reviews from the positive foler and Class 0 represents reviews from negative folder

### Using Regex to clean up the data

In [55]:
#https://stackoverflow.com/questions/9662346/python-code-to-remove-html-tags-from-a-string

import re

def cleanhtml(raw_html):
    cleanr = re.compile('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

### Splitting the dataset

In [56]:
#imports:

from sklearn.model_selection import train_test_split
import numpy as np

In [57]:
#x = df.iloc[:,1:2] #Review column
#y = df["Class"] #Class column

#print(x.size)

#80/20 split
#x_develop,x_train,y_develop,y_train=train_test_split(x,y,test_size=0.2,random_state=np.random)

train,develop  = train_test_split(df, test_size=0.2)

type(train)
print(train.shape)
print(develop.shape)
print(test_df.shape)

(20000, 3)
(5000, 3)
(25000, 3)


### Vocabulary List for all occurences

In [130]:
#https://www.w3resource.com/python-exercises/string/python-data-type-string-exercise-12.php

def new_word_count(df):
    counts = dict()
    counts.update( {'total_pos' : 0} )
    counts.update( {'total_neg' : 0} )
    for index, row in df.iterrows():
        #print (row["Review"])
        row["Review"] = cleanhtml(row["Review"])
        row["Review"] = re.sub('\S+', lambda m: re.sub('^\W+|\W+$', '', m.group()), row["Review"])
        row["Review"] = row["Review"].lower()
        words = row["Review"].split()
        explored_words = []
        if row['Class']=='1':
            counts['total_pos'] += 1
        else:
            counts['total_neg'] += 1
        for word in words:
            if word in counts:
                if word in explored_words:
                    continue
                else:    
                    counts[word] += 1
                    explored_words.append(word)
            else:
                counts[word] = 1
                explored_words.append(word)
            
    return counts

    
    
train_vocList = new_word_count(train)
develop_vocList = new_word_count(develop)
test_vocList = new_word_count(test_df)
#train_vocList

In [131]:
#https://thispointer.com/python-filter-a-dictionary-by-conditions-on-keys-or-values/
#https://kite.com/python/answers/how-to-remove-multiple-keys-from-a-dictionary-in-python
import copy

def filterRare(vocList,minOccurence):
    newList = copy.deepcopy(vocList)
    keys_to_remove = []
    for (key, value) in newList.items():
       # Check if key is even then add pair to new dictionary
       if value < minOccurence:
            keys_to_remove.append(key)
    
    #print(keys_to_remove)        
    
    for key in keys_to_remove:
        del newList[key]

    return newList

    #print('Filtered Dictionary : ')
    #print(newDict)

In [132]:
print("No. of Words in original train dictionary: "+str(len(train_vocList)))
train_vocList = filterRare(train_vocList,5);
print("No. of Words, in train, which have frequency > 5: "+str(len(train_vocList)))
print("No. of Words in original develop dictionary: "+str(len(develop_vocList)))
develop_vocList = filterRare(develop_vocList,5);
print("No. of Words, in develop, which have frequency > 5: "+str(len(develop_vocList)))
print("No. of Words in original test dictionary: "+str(len(test_vocList)))
test_vocList = filterRare(test_vocList,5);
print("No. of Words,in test, which have frequency > 5: "+str(len(test_vocList)))

No. of Words in original train dictionary: 107183
No. of Words, in train, which have frequency > 5: 26694
No. of Words in original develop dictionary: 51042
No. of Words, in develop, which have frequency > 5: 11660
No. of Words in original test dictionary: 118698
No. of Words,in test, which have frequency > 5: 29556


### Probability of the occurrence

In [174]:
def calc_probability(word,dataset,vocList):          
    prob = vocList[word]/len(dataset)
    print("P["+word+"]: "+str(vocList[word])+"/"+str(len(dataset))+"="+str(prob))
    return prob

#P[“the”] = num of documents containing ‘the’ / num of all documents
calc_probability("the",train,train_vocList)

def calc_probability_smoothing(word,dataset,vocList, laplaceVal):
    prob_smooth = (vocList[word] + laplaceVal) / (len(dataset) + len(vocList) * laplaceVal)
    print("P["+word+"]: "+str(vocList[word]+ laplaceVal)+"/"+str(len(dataset)+ len(vocList) * laplaceVal)+", with smoothing= "+str(prob_smooth))
    return prob_smooth

P[the]: 19840/20000=0.992


### Conditional probability based on the sentiment

In [182]:
def conditional_probability(cond_word, dataset, vocList,condition):
    count = 0
    for index, row in dataset.iterrows():
        words_in_review = row["Review"].split()
        for word in words_in_review:
            if(word == cond_word and row["Class"]== condition ):
                count +=1
                break
    target = "Positive"
    if (condition == '1'):
        total = vocList['total_pos']
    else:
        total = vocList['total_neg']
        target = "Negative"
    
    condProb = count/total
    print("P["+cond_word+"|"+target+" ]: "+str(count)+"/"+str(total)+"= "+str(condProb))
    
    return condProb


#P[“the” | Positive]  = # of positive documents containing “the” / num of all positive review documents
conditional_probability('the',train,train_vocList,'1')


def conditional_probability_smoothing(cond_word, dataset, vocList,condition,laplaceVal):
    count = 0
    for index, row in dataset.iterrows():
        words_in_review = row["Review"].split()
        for word in words_in_review:
            if(word == cond_word and row["Class"]== condition ):
                count +=1
                break
    target = "Positive"
    if (condition == '1'):
        total = vocList['total_pos']
    else:
        total = vocList['total_neg']
        target = "Negative"
    
    sCondProb = (count+laplaceVal)/(total + laplaceVal * len(vocList))
    print("P["+cond_word+"|"+target+" ]: "+str(count+laplaceVal)+"/"+str(total + laplaceVal * len(vocList))+", with smoothing= "+str(sCondProb))
    
    return sCondProb

P[the|Positive ]: 9907/9995= 0.9911955977988994


### Naive bayes Classifier

In [186]:
#P(Class|word) = (P(Class)*P(word|Class))/P(word)
def classifier(vocList,condition, cWord, dataset):
    target = 'Positive'
    if (condition == '1'):
        totalClass = vocList['total_pos']
    else:
        target = 'Negative'
        totalClass = vocList['total_neg']
    
    classProbability = totalClass/ len(dataset)
    #print("Probability of Class "+target +": "+str(classProbability))
    
    condProb = conditional_probability(cWord,dataset,vocList,condition)
    
    probabilityWord = calc_probability(cWord,dataset,vocList)
    
    prob_class_given_word = (classProbability * condProb)/probabilityWord
    print("P["+target+"|"+cWord+"]: "+str(prob_class_given_word))
    
    return classProbability
    
    

classifier(train_vocList,'1','the',train)
print("----------------------------------------------------")
classifier(train_vocList,'0','the',train)


#P(Class|word) = (P(Class)*P(word|Class))/P(word)
def classifier_smoothing(vocList,condition, cWord, dataset,laplaceVal):
    target = 'Positive'
    if (condition == '1'):
        totalClass = vocList['total_pos']
    else:
        target = 'Negative'
        totalClass = vocList['total_neg']
    
    classProbability = (totalClass+ laplaceVal)/ (len(dataset) + laplaceVal*2)
    #print("Probability of Class "+target +": "+str(classProbability))
    
    condProb = conditional_probability_smoothing(cWord,dataset,vocList,condition,laplaceVal)
    
    probabilityWord = calc_probability_smoothing(cWord,dataset,vocList,laplaceVal)
    
    sProb_class_given_word = (classProbability * condProb)/probabilityWord
    print("P["+target+"|"+cWord+"]: "+str(sProb_class_given_word))
    
    return classProbability

P[the|Positive ]: 9907/9995= 0.9911955977988994
P[the]: 19840/20000=0.992
P[Positive|the]: 0.49934475806451617
----------------------------------------------------
P[the|Negative ]: 9933/10005= 0.9928035982008996
P[the]: 19840/20000=0.992
P[Negative|the]: 0.5006552419354838


### Accuracy using dev dataset 

####  Five fold cross validation

In [136]:
def cross_validation(dataset,cWord):
    #print(dataset.shape[0])
    foldSize = int(len(dataset)/5)
    #print (foldSize)
    #dataset_np = dataset.to_numpy()
    d1 = dataset[0:foldSize]
    #print(d1["Review"])
    d2 = dataset[foldSize:foldSize+1000]
    
    foldSize = foldSize + 1000
    d3 = dataset[foldSize:foldSize+1000]
    
    foldSize = foldSize + 1000
    d4 = dataset[foldSize:foldSize+1000]
    
    foldSize = foldSize + 1000
    d5 = dataset[foldSize:foldSize+1000]
    accuracies = []
    words_in_review = []
    folds = [d1,d2,d3,d4,d5]
    for chunk in folds:
        vocList = new_word_count(chunk)
        print("-------------------------------------------------------------")
        print("No. of Words in dataset vocaublary list: "+str(len(vocList)))
        vocList = filterRare(vocList,5)
        print("No. of Words, which have frequency > 5: "+str(len(vocList)))
        probability = classifier(vocList,'1',cWord,chunk)
        target = '1' #positive sentiment
        if probability < 0.50:
            target = '0' #negative sentiment
        correct = 0
        #print(chunk)
        for index, row in chunk.iterrows():
            words_in_review = row["Review"].split()
            for word in words_in_review:
                if(word == cWord and row["Class"] == target):
                    correct +=1
                    break
        #print(correct)
        #print(chunk.shape[0])
        accuracies.append(correct/chunk.shape[0])
        
    
    #print(accuracies)
    
    return (sum(accuracies) / len(accuracies))

print("K-fold cross validation with K as 5:"+str(cross_validation(develop,'the')))

-------------------------------------------------------------
No. of Words in dataset vocaublary list: 21206
No. of Words, which have frequency > 5: 3627
P[the|Positive ]: 488/492= 0.991869918699187
P[the]: 993/1000=0.993
P[Positive|the]: 0.4914400805639476
-------------------------------------------------------------
No. of Words in dataset vocaublary list: 20498
No. of Words, which have frequency > 5: 3516
P[the|Positive ]: 486/492= 0.9878048780487805
P[the]: 991/1000=0.991
P[Positive|the]: 0.49041372351160445
-------------------------------------------------------------
No. of Words in dataset vocaublary list: 21381
No. of Words, which have frequency > 5: 3705
P[the|Positive ]: 508/520= 0.9769230769230769
P[the]: 983/1000=0.983
P[Positive|the]: 0.5167853509664293
-------------------------------------------------------------
No. of Words in dataset vocaublary list: 21206
No. of Words, which have frequency > 5: 3786
P[the|Positive ]: 489/494= 0.9898785425101214
P[the]: 992/1000=0.992


#### Effects of Smoothing

In [187]:
#https://medium.com/syncedreview/applying-multinomial-naive-bayes-to-nlp-problems-a-practical-explanation-4f5271768ebf

calc_probability("the",train,train_vocList)
calc_probability_smoothing("the",train,train_vocList,1)
print("-----------------------------------------------")
conditional_probability('the',train,train_vocList,'1')
conditional_probability_smoothing('the',train,train_vocList,'1',1)
print("-----------------------------------------------")
classifier(train_vocList,'1','the',train)
classifier_smoothing(train_vocList,'1','the',train,1)

P[the]: 19840/20000=0.992
P[the]: 19841/46694, with smoothing= 0.4249154066903671
-----------------------------------------------
P[the|Positive ]: 9907/9995= 0.9911955977988994
P[the|Positive ]: 9908/36689, with smoothing= 0.2700536945678541
-----------------------------------------------
P[the|Positive ]: 9907/9995= 0.9911955977988994
P[the]: 19840/20000=0.992
P[Positive|the]: 0.49934475806451617
P[the|Positive ]: 9908/36689, with smoothing= 0.2700536945678541
P[the]: 19841/46694, with smoothing= 0.4249154066903671
P[Positive|the]: 0.31761460866326346


0.49975002499750026

#### Top 10 words that predict positive and negative class

In [ ]:
#P[Positive| word] 

#using top 1000 words to find the top10 predictors of classes
#this can be skipped, and the classifier can be run on entire vocabulary list, but that is time intensive task
tempVocList = filterRare(train_vocList,400)
posProbs = {}
print(len(tempVocList))
probability = 0

for (key, value) in tempVocList.items():
    #print(type(key))
    #print(key)
    probability = classifier(tempVocList,'1',key,train) #classifier(vocList,condition, cWord, dataset):
    posProbs.update( {key : probability} )

type(posProbs.items())
#--------------------------------
negProbs = {}

probability = 0
for (key, value) in tempVocList.items():
    #print(type(key))
    #print(key)
    probability = classifier(tempVocList,'0',key,train) #classifier(vocList,condition, cWord, dataset):
    negProbs.update( {key : probability} )

##This was a time intensive task, even after filtering out words with less than 8000times occurence, i still had 40/50 words
## which took considerable time to calculate.
##However, given enough time it will calculate the top 1000 words


#https://www.w3resource.com/python-exercises/dictionary/python-data-type-dictionary-exercise-1.php
#Sorting posProbs & negProbs
sorted_posProbs = dict(sorted(posProbs.items(), key=operator.itemgetter(1),reverse=True))
#print('Dictionary in descending order by value : ',sorted_posProbs)

sorted_negProbs = dict(sorted(negProbs.items(), key=operator.itemgetter(1),reverse=True))
#print('Dictionary in descending order by value : ',sorted_negProbs)

#https://stackoverflow.com/questions/29216889/slicing-a-dictionary
import itertools

posProbs_top10 = dict(itertools.islice(posProbs.items(), 10))
negProbs_top10 = dict(itertools.islice(negProbs.items(), 10))

print("Top 10 predictors for the positive class")
print(posProbs_top10)

print("Top 10 predictors for the negative class")
print(negProbs_top10)